In [1]:
import pandas as pd
import numpy as np
import joblib

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score
from sklearn import tree
from nltk.corpus import stopwords

In [5]:
#для устранения ошибки "Columns (3) have mixed types" ставим low_memory=False
df = pd.read_csv('lenta-ru-news.csv', low_memory=False)

In [6]:
df

,url,title,text,topic,tags,date
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая,1914/09/16
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая,1914/09/16
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая,1914/09/17
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая,1914/09/17
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая,1914/09/18
...,...,...,...,...,...,...
800970,https://lenta.ru/news/2019/12/14/shnur/,Шнуров раскритиковал Гагарину на «Голосе»,Певец Сергей Шнуров раскритиковал свою коллегу...,NaN,ТВ и радио,2019/12/14
800971,https://lenta.ru/news/2019/12/14/dolg/,В России предложили изменить правила взыскания...,Министерство юстиции России предложило изменит...,NaN,Все,2019/12/14
800972,https://lenta.ru/news/2019/12/14/dark_euro/,В России назвали «черную дату» для Европы,Испытание США ранее запрещенной Договором о ли...,NaN,Политика,2019/12/14
800973,https://lenta.ru/news/2019/12/14/meteo/,Россиянам пообещали аномально теплую погоду,В ближайшие дни в европейской части России пог...,NaN,Общество,2019/12/14


In [7]:
df.topic.unique()

array(['Библиотека', 'Россия', 'Мир', 'Экономика', 'Интернет и СМИ',
       'Спорт', 'Культура', 'Из жизни', 'Силовые структуры',
       'Наука и техника', 'Бывший СССР', nan, 'Дом', 'Сочи', 'ЧМ-2014',
       'Путешествия', 'Ценности', 'Легпром', 'Бизнес', 'МедНовости',
       'Оружие', '69-я параллель', 'Культпросвет ', 'Крым'], dtype=object)

In [8]:
topics_list = ['Россия', 'Мир', 'Экономика', 'Спорт', 'Культура']

In [9]:
#оставляем только новости по нужным темам
df = df[df.topic.isin(topics_list)]

In [10]:
df.topic.unique()

array(['Россия', 'Мир', 'Экономика', 'Спорт', 'Культура'], dtype=object)

In [11]:
df.shape

(494804, 6)

In [12]:
df.isna().sum()

url          0
title        0
text         5
topic        0
tags     15328
date         0
dtype: int64

In [13]:
df = df.copy()
df.dropna(subset=['text'], inplace = True)
df.shape

(494799, 6)

In [14]:
df.drop(columns=['url', 'tags', 'date'], axis=1, inplace=True)

In [15]:
df

,title,text,topic
5,"Космонавты сомневаются в надежности ""Мира""",Как стало известно агентству Ассошиэйтед Пресс...,Россия
6,Взрыв в центре Москвы: пострадало 30 человек,В зале игровых автоматов в третьем ярусе подзе...,Россия
7,Япония кредитует Россию на полтора миллиарда д...,Япония приняла решение разморозить кредиты Рос...,Россия
8,Британцы отмечают двухлетие смерти Дианы,Британцы отмечают сегодня скорбную дату - втор...,Мир
9,Отмытые через Bank of NY деньги не имели отнош...,В понедельник директор департамента внешних св...,Россия
...,...,...,...
739168,Путин предостерег от запретов рэп-концертов,"Президент России Владимир Путин, выступая на з...",Культура
739169,Путин подарил тяжелобольному мальчику обещанны...,Президент России Владимир Путин организовал дл...,Россия
739171,«Желтые жилеты» прорвали кордон полиции под «М...,Протесты движения «желтых жилетов» в Париже сн...,Мир
739174,Падение горнолыжника на полной скорости попало...,Швейцарский горнолыжник Марк Гизин неудачно пр...,Спорт


In [16]:
#преобразование текста (получаем матрицу слов, метод TF-IDF) 
text_transformer = TfidfVectorizer()
text = text_transformer.fit_transform(df['text'])

In [18]:
text.shape

(494799, 821732)

In [17]:
#разделим сет текстов, на тестовые данные отводится 30%
x_train, x_test, y_train, y_test = train_test_split(text, df['topic'], test_size=0.3, random_state=696)

In [19]:
%%time
#обучение
clf = tree.DecisionTreeClassifier()
clf.fit(x_train, y_train)

CPU times: total: 49min 40s
Wall time: 49min 41s


DecisionTreeClassifier()

In [23]:
pred = clf.predict(x_test)
print('Качество модели по метрике F1 без обработки =', np.round(f1_score(y_test, pred, average='weighted'), decimals=2))

Качество модели по метрике F1 без обработки = 0.79


In [24]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [25]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Дружбанчик\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [26]:
stop_russian = stopwords.words('russian')

In [27]:
#функция удаления из текста небуквенных символов
import re
def remove_trash(list):
    pattern = r'[^А-Яа-я]+' #1 и более небуквенных символов
    try:
      list = [re.sub(pattern, ' ', i) for i in list] 
    except Exception as e:
      print(e)
    return list

In [28]:
%%time
df['text_clean'] = remove_trash(df['text'])

CPU times: total: 16.5 s
Wall time: 16.5 s


In [29]:
df.head()

,title,text,topic,text_clean
5,"Космонавты сомневаются в надежности ""Мира""",Как стало известно агентству Ассошиэйтед Пресс...,Россия,Как стало известно агентству Ассошиэйтед Пресс...
6,Взрыв в центре Москвы: пострадало 30 человек,В зале игровых автоматов в третьем ярусе подзе...,Россия,В зале игровых автоматов в третьем ярусе подзе...
7,Япония кредитует Россию на полтора миллиарда д...,Япония приняла решение разморозить кредиты Рос...,Россия,Япония приняла решение разморозить кредиты Рос...
8,Британцы отмечают двухлетие смерти Дианы,Британцы отмечают сегодня скорбную дату - втор...,Мир,Британцы отмечают сегодня скорбную дату вторую...
9,Отмытые через Bank of NY деньги не имели отнош...,В понедельник директор департамента внешних св...,Россия,В понедельник директор департамента внешних св...


In [30]:
df.isna().sum()

title         0
text          0
topic         0
text_clean    0
dtype: int64

In [31]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
#функция лемматизации (приведение к первой форме слова), слова из 1-2 букв выбрасываем
def lemmatize(row):
    t = []
    text = row['text_clean']
    for word in text.split():
        if len(word)<=2:
            continue
        p = morph.parse(word)[0]
        t.append(p.normal_form)
    return " ".join(t)

In [32]:
%%time
#обработка текстов (лемматизация)
df['text_clean_normal'] = df.apply(lemmatize, axis=1)

CPU times: total: 1h 40min 25s
Wall time: 1h 40min 28s


In [33]:
df.head()

,title,text,topic,text_clean,text_clean_normal
5,"Космонавты сомневаются в надежности ""Мира""",Как стало известно агентству Ассошиэйтед Пресс...,Россия,Как стало известно агентству Ассошиэйтед Пресс...,как стать известно агентство ассошиэйтед пресс...
6,Взрыв в центре Москвы: пострадало 30 человек,В зале игровых автоматов в третьем ярусе подзе...,Россия,В зале игровых автоматов в третьем ярусе подзе...,зал игровой автомат третий ярус подземный комп...
7,Япония кредитует Россию на полтора миллиарда д...,Япония приняла решение разморозить кредиты Рос...,Россия,Япония приняла решение разморозить кредиты Рос...,япония принять решение разморозить кредит росс...
8,Британцы отмечают двухлетие смерти Дианы,Британцы отмечают сегодня скорбную дату - втор...,Мир,Британцы отмечают сегодня скорбную дату вторую...,британец отмечать сегодня скорбный дата вторую...
9,Отмытые через Bank of NY деньги не имели отнош...,В понедельник директор департамента внешних св...,Россия,В понедельник директор департамента внешних св...,понедельник директор департамент внешний связь...


In [34]:
%%time
#проведем разделение выборки и обучение на обработанных данных
text_transformer = TfidfVectorizer(stop_words=stop_russian, 
                                   ngram_range=(1,1), #будем брать последовательность из одного слова
                                   lowercase=True)
text_norm = text_transformer.fit_transform(df['text_clean_normal'])
x_train, x_test, y_train, y_test = train_test_split(text_norm, df['topic'], test_size=0.3, random_state=random_state)

CPU times: total: 34.3 s
Wall time: 34.3 s


In [35]:
%%time
clf_norm = tree.DecisionTreeClassifier()
clf_norm.fit(x_train, y_train)

CPU times: total: 27min 35s
Wall time: 27min 35s


DecisionTreeClassifier()

In [36]:
pred_norm = clf_norm.predict(x_test)
print('Качество модели по метрике F1 после обработки текста = ', 
      np.round(f1_score(y_test, pred_norm, average='weighted'), decimals=2))

Качество модели по метрике F1 после обработки текста =  0.81
